# Transformer Block and Attention Mechanism

glossary:

- shortcut: mitigate vanishing gradiant
- normalization: improve training efficiency
- attention: token influence in context
- dropout: random removal to mitigate overfitting 

In [11]:
import torch
import torch.nn as nn
import myllm.attention as att
import myllm.layers as layers
import myllm.transformer as transformer



In [19]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = att.MultiHeadAttention(
            d_in = cfg["emb_dim"],
            d_out = cfg["emb_dim"],
            num_heads = cfg["n_heads"],
            dropout = cfg["drop_rate"],
            qkv_bias = cfg["qkv_bias"],
            context_length = cfg["context_length"],
        )
        self.ff = layers.FeedForward(cfg)
        self.norm1 = layers.LayerNorm(cfg["emb_dim"])
        self.norm2 = layers.LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)

        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

### Application

In [20]:
torch.manual_seed(123)
x = torch.rand(2,4,768)

block = TransformerBlock(transformer.GPT_CONFIG_124M)

output = block(x)

print("input", x.shape)
print("output", output.shape)

input torch.Size([2, 4, 768])
output torch.Size([2, 4, 768])


## The GPT Model

In [26]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # define embeddings 
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm = layers.LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb( torch.arange(seq_len, device=in_idx.device))

        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)

        logits = self.out_head(x)
        return logits



In [27]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch=[]

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))

batch = torch.stack(batch, dim=0)

In [28]:

torch.manual_seed(123)
model = GPTModel(transformer.GPT_CONFIG_124M)

out = model(batch)

print("input: ", batch)
print("output: ", out.shape)
print(out)

input:  tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
output:  torch.Size([2, 4, 50257])
tensor([[[ 0.4100,  0.0773, -0.1233,  ...,  0.4064,  0.1511, -0.2058],
         [ 0.2380, -0.6609, -0.8712,  ..., -0.2786,  0.3351, -0.1805],
         [ 1.0871,  0.1384, -0.3017,  ...,  0.0142, -0.5550, -0.1027],
         [-0.7906,  0.5643, -0.1533,  ...,  0.7284,  0.3933,  0.0907]],

        [[-0.4984, -0.1907, -0.0464,  ...,  0.2285,  0.0701, -0.3359],
         [ 0.2858,  0.3912, -0.1629,  ...,  0.7203, -0.1694,  0.3580],
         [ 1.0323,  0.7883, -0.1233,  ...,  0.7404,  0.1473, -0.1822],
         [-0.1140,  0.5510,  0.2920,  ...,  1.2281, -0.1607, -0.0441]]],
       grad_fn=<UnsafeViewBackward0>)


In [34]:
total_params = sum(p.numel() for p in model.parameters())

print("total parameters", total_params)

# why the size is bigger than 124M?
print("token emb layer", model.tok_emb.weight.shape)
print("output layer shape", model.out_head.weight.shape)

# weight tying
total_params_gpt2 = total_params - sum(p.numel() for p in model.out_head.parameters())
print("total params considering weight tying: ", total_params_gpt2)

total parameters 163009536
token emb layer torch.Size([50257, 768])
output layer shape torch.Size([50257, 768])
total params considering weight tying:  124412160
